# Connecting to griddap datasets

This notebook shows the available functionality in [ERDDAP_Griddap](https://hmedrano.github.io/erddap-python/#ERDDAP_Griddap) Class to get information from remote griddap datasets.



In [1]:
# Create the griddap object

from erddapClient import ERDDAP_Griddap

remote = ERDDAP_Griddap('https://coastwatch.pfeg.noaa.gov/erddap','hycom_gom310D')

# Show me the dataset general information (Until here, remote data is requested)
print(remote)

<erddapClient.ERDDAP_Griddap>
Title:       NRL HYCOM 1/25 deg model output, Gulf of Mexico, 10.04 Expt 31.0, 2009-2014, At Depths
Server URL:  https://coastwatch.pfeg.noaa.gov/erddap
Dataset ID:  hycom_gom310D
Dimensions: 
  time (double) range=(cftime.DatetimeGregorian(2009, 4, 2, 0, 0, 0, 0), cftime.DatetimeGregorian(2014, 8, 30, 0, 0, 0, 0)) 
    Standard name: time 
    Units:         seconds since 1970-01-01T00:00:00Z 
  depth (float) range=(0.0, 5500.0) 
    Standard name: depth 
    Units:         m 
  latitude (float) range=(18.09165, 31.96065) 
    Standard name: latitude 
    Units:         degrees_north 
  longitude (float) range=(-98.0, -76.40002) 
    Standard name: longitude 
    Units:         degrees_east 
Variables: 
  temperature (float) 
    Standard name: sea_water_potential_temperature 
    Units:         degC 
  salinity (float) 
    Standard name: sea_water_practical_salinity 
    Units:         psu 
  u (float) 
    Standard name: eastward_sea_water_velocity 
  

## Exploring the dimensions

The dimension information of the dataset is downloaded and parsed. Property
and methods are provided to help in the exploration.

More on dimensions methods [ERDDAP Dimensions](https://hmedrano.github.io/erddap-python/#ERDDAP_Griddap_dimension)

In [2]:
print(remote.dimensions)

<erddapClient.ERDDAP_Griddap_dimensions>
Dimensions:
 - time (nValues=1977) 1238630400 .. 1409356800
 - depth (nValues=40) 0.0 .. 5500.0
 - latitude (nValues=385) 18.091648 .. 31.960648
 - longitude (nValues=541) -98.0 .. -76.400024


In [3]:
print(remote.dimensions['depth'])

<erddapClient.ERDDAP_Griddap_dimension>
Dimension: depth
  _nValues : 40
  _evenlySpaced : False
  _averageSpacing : 141.02564102564102
  _dataType : float
  _CoordinateAxisType : Height
  _CoordinateZisPositive : down
  actual_range : (0.0, 5500.0)
  axis : Z
  ioos_category : Location
  long_name : Depth
  positive : down
  standard_name : depth
  units : m


In [4]:
# Each dimension, has a dictionary info with the 
# variable dimension attributes
remote.dimensions['depth'].info['units']

'm'

In [5]:
# The property data hold the dimension values
remote.dimensions['depth'].data

Float64Index([   0.0,    5.0,   10.0,   15.0,   20.0,   25.0,   30.0,   40.0,
                50.0,   60.0,   70.0,   80.0,   90.0,  100.0,  125.0,  150.0,
               200.0,  250.0,  300.0,  400.0,  500.0,  600.0,  700.0,  800.0,
               900.0, 1000.0, 1100.0, 1200.0, 1300.0, 1400.0, 1500.0, 1750.0,
              2000.0, 2500.0, 3000.0, 3500.0, 4000.0, 4500.0, 5000.0, 5500.0],
             dtype='float64')

In [6]:
# The time variable dimension, has a special attribute timeData that
# has the values parsed to python datetimes
remote.dimensions['time'].timeData

array([cftime.DatetimeGregorian(2009, 4, 2, 0, 0, 0, 0),
       cftime.DatetimeGregorian(2009, 4, 3, 0, 0, 0, 0),
       cftime.DatetimeGregorian(2009, 4, 4, 0, 0, 0, 0), ...,
       cftime.DatetimeGregorian(2014, 8, 28, 0, 0, 0, 0),
       cftime.DatetimeGregorian(2014, 8, 29, 0, 0, 0, 0),
       cftime.DatetimeGregorian(2014, 8, 30, 0, 0, 0, 0)], dtype=object)

In [7]:
# Find closest integer index of a dimension value
remote.dimensions['time'].closestIdx('2010-06-15')

439

## Exploring metadata

Methods and properties to explore the metadata

In [8]:
# The object will parse the metadata from the dataset, and store it in the info property
remote.info['summary']

'NRL HYCOM 1/25 deg model output, Gulf of Mexico, 10.04 Expt 31.0, 2009-2014, At Depths\n\nThe HYCOM consortium is a multi-institutional effort sponsored by the\nNational Ocean Partnership Program (NOPP), as part of the U. S. Global Ocean\nData Assimilation Experiment (GODAE), to develop and evaluate a\ndata-assimilative hybrid isopycnal-sigma-pressure (generalized) coordinate\nocean model (called HYbrid Coordinate Ocean Model or HYCOM).'

In [9]:
# You could also use the getAttribute method
remote.getAttribute('time_coverage_start')

'2009-04-02T00:00:00Z'

In [10]:
# The getAttribute method can specify the variable from where to get a metadata 
# attribute. By defaults searches from the global metadata
remote.getAttribute('standard_name', 'salinity')

'sea_water_practical_salinity'

## Metadata from dataset variables

Methods and properties to get variable information

In [11]:
# List variables of dataset
remote.variables.keys()

odict_keys(['temperature', 'salinity', 'u', 'v', 'w_velocity'])

In [12]:
remote.variables['u']

{'_dataType': 'float',
 '_FillValue': 1.267651e+30,
 'colorBarMaximum': 0.5,
 'colorBarMinimum': -0.5,
 'ioos_category': 'Currents',
 'long_name': 'Eastward Sea Water Velocity',
 'standard_name': 'eastward_sea_water_velocity',
 'units': 'm/s',
 'valid_range': (-1.0337982, 2.039324)}

In [13]:
remote.getAttribute('units','u')

'm/s'

## Requesting data from ERDDAP Server

The library can also help you build the url's to make metadata and data requests. 



In [24]:
# Start by clearing the query stack
remote.clearQuery()

# Select the variables from which you want results,
# in erddap terms, the 'result variables'
remote.setResultVariables(['temperature', 'salinity'])

# Now, set the subset you want to get, by filtering the 
# dimensions.
# This method accepts the dimensions name, and the specific value to request,
# or a python slice
remote.setSubset(time='2009-06-15',
                 depth=0,
                 latitude=21.5,
                 longitude=-81.4)

# Show me the url to get the above request
remote.getURL('htmlTable')

'https://coastwatch.pfeg.noaa.gov/erddap/griddap/hycom_gom310D.htmlTable?temperature%5B74%3A74%5D%5B0%3A0%5D%5B91%3A91%5D%5B415%3A415%5D%2Csalinity%5B74%3A74%5D%5B0%3A0%5D%5B91%3A91%5D%5B415%3A415%5D'

In [25]:
from IPython.core.display import display, HTML
# Request the data
HTML(remote.getData('htmlTable'))

,ERDDAP Easier access to scientific data,log in Brought to you by NOAA NMFS SWFSC ERD


In [27]:
# Request the data in another format
print(remote.getData('csvp'))

time (UTC),depth (m),latitude (degrees_north),longitude (degrees_east),temperature (degC),salinity (psu)
2009-06-15T00:00:00Z,0.0,21.515734,-81.400024,29.730402,36.1295



In [28]:
# Request a DataFrame
remote.getDataFrame()

,time (UTC),depth (m),latitude (degrees_north),longitude (degrees_east),temperature (degC),salinity (psu)
0,2009-06-15T00:00:00Z,0.0,21.515734,-81.400024,29.730402,36.1295


### Chain the commands

The above request can be made in a chained style.

In [29]:
# Clear the query stack
remote.clearQuery()

# Select the variables from which you want results,
# in erddap terms, the 'result variables'
responseDF = (
    remote.setResultVariables(['temperature', 'salinity']) 
          .setSubset(time=slice('2009-06-15','2009-07-15'),
                     depth=0,
                     latitude=21.5,
                     longitude=-81.4)
          .getDataFrame())

responseDF

,time (UTC),depth (m),latitude (degrees_north),longitude (degrees_east),temperature (degC),salinity (psu)
0,2009-06-15T00:00:00Z,0.0,21.515734,-81.400024,29.730402,36.129500
1,2009-06-16T00:00:00Z,0.0,21.515734,-81.400024,29.489584,36.131210
2,2009-06-17T00:00:00Z,0.0,21.515734,-81.400024,29.621000,36.120747
3,2009-06-18T00:00:00Z,0.0,21.515734,-81.400024,29.944820,36.100903
4,2009-06-19T00:00:00Z,0.0,21.515734,-81.400024,29.970623,36.094010
5,2009-06-20T00:00:00Z,0.0,21.515734,-81.400024,30.491571,36.106026
6,2009-06-21T00:00:00Z,0.0,21.515734,-81.400024,30.555803,36.096780
7,2009-06-22T00:00:00Z,0.0,21.515734,-81.400024,31.188433,36.092960
8,2009-06-23T00:00:00Z,0.0,21.515734,-81.400024,30.556208,36.089622
9,2009-06-24T00:00:00Z,0.0,21.515734,-81.400024,30.257763,36.069275


### Request a 3D subset, in a xarray object

If the subset you're requesting is 2D o 3D, you can get a xarray object with only
this subset, connected to the erddap server thru the opendap service.  


In [32]:
# Clear the query stack
remote.clearQuery()

# Select the variables from which you want results,
# in erddap terms, the 'result variables'
responseX = (
    remote.setResultVariables(['temperature', 'salinity'])
          .setSubset(time='2009-06-15',
                     depth=0,
                     latitude=slice(21.5,28.4),
                     longitude=slice(-85.4, -78.5))
          .getxArray())

responseX

<xarray.Dataset>
Dimensions:      (depth: 1, latitude: 191, longitude: 174, time: 1)
Coordinates:
  * time         (time) object 2009-06-15 00:00:00
  * depth        (depth) float64 0.0
  * latitude     (latitude) float64 21.52 21.55 21.59 21.63 ... 28.33 28.36 28.4
  * longitude    (longitude) float64 -85.4 -85.36 -85.32 ... -78.52 -78.48
Data variables:
    temperature  (time, depth, latitude, longitude) float32 ...
    salinity     (time, depth, latitude, longitude) float32 ...
Attributes: (12/32)
    cdm_data_type:              Grid
    Conventions:                COARDS, CF-1.0, ACDD-1.3
    creator_email:              hycomdata@coaps.fsu.edu
    creator_name:               Naval Research Laboratory
    creator_type:               institution
    creator_url:                https://www.hycom.org
    ...                         ...
    standard_name_vocabulary:   CF Standard Name Table v70
    summary:                    NRL HYCOM 1/25 deg model output, Gulf of Mexi...
    time_coverage_end:          2014-08-30T00:00:00Z
    time_coverage_start:        2009-04-02T00:00:00Z
    title:                      NRL HYCOM 1/25 deg model output, Gulf of Mexi...
    Westernmost_Easting:        -98.0

### Request a subset by the integer indexes

In the above examples we, select our subset by the dimension values, but the ERDDAP_Griddap 
object also defines a method to subset by the integer indexes of the dimensions. `setSubsetI`



In [33]:
# Clear the query stack
remote.clearQuery()

# Select the variables from which you want results,
# in erddap terms, the 'result variables'
responseX = (
    remote.setResultVariables(['temperature', 'salinity'])
          .setSubsetI(time=10,
                      depth=0,
                      latitude=slice(0,20),
                      longitude=slice(0,30))
          .getxArray())

responseX

<xarray.Dataset>
Dimensions:      (depth: 1, latitude: 20, longitude: 30, time: 1)
Coordinates:
  * time         (time) object 2009-04-12 00:00:00
  * depth        (depth) float64 0.0
  * latitude     (latitude) float64 18.09 18.13 18.17 ... 18.74 18.77 18.81
  * longitude    (longitude) float64 -98.0 -97.96 -97.92 ... -96.88 -96.84
Data variables:
    temperature  (time, depth, latitude, longitude) float32 ...
    salinity     (time, depth, latitude, longitude) float32 ...
Attributes: (12/32)
    cdm_data_type:              Grid
    Conventions:                COARDS, CF-1.0, ACDD-1.3
    creator_email:              hycomdata@coaps.fsu.edu
    creator_name:               Naval Research Laboratory
    creator_type:               institution
    creator_url:                https://www.hycom.org
    ...                         ...
    standard_name_vocabulary:   CF Standard Name Table v70
    summary:                    NRL HYCOM 1/25 deg model output, Gulf of Mexi...
    time_coverage_end:          2014-08-30T00:00:00Z
    time_coverage_start:        2009-04-02T00:00:00Z
    title:                      NRL HYCOM 1/25 deg model output, Gulf of Mexi...
    Westernmost_Easting:        -98.0

### Request a subset, by writting the subset string

The method `setResultVariables` can accept the subset string, in the extended opendap format
that ERDDAP implements.

In [38]:
# Clear the query stack
remote.clearQuery()

# Select the variables from which you want results,  but also you can include 
# the subset string
responseX = (
    remote.setResultVariables("temperature[(2009-04-12)][0][(18.10):(18.81)][(-98.0):(-96.84)]")
          .getxArray())

responseX

<xarray.Dataset>
Dimensions:      (depth: 1, latitude: 20, longitude: 30, time: 1)
Coordinates:
  * time         (time) object 2009-04-12 00:00:00
  * depth        (depth) float64 0.0
  * latitude     (latitude) float64 18.09 18.13 18.17 ... 18.74 18.77 18.81
  * longitude    (longitude) float64 -98.0 -97.96 -97.92 ... -96.88 -96.84
Data variables:
    temperature  (time, depth, latitude, longitude) float32 ...
Attributes: (12/32)
    cdm_data_type:              Grid
    Conventions:                COARDS, CF-1.0, ACDD-1.3
    creator_email:              hycomdata@coaps.fsu.edu
    creator_name:               Naval Research Laboratory
    creator_type:               institution
    creator_url:                https://www.hycom.org
    ...                         ...
    standard_name_vocabulary:   CF Standard Name Table v70
    summary:                    NRL HYCOM 1/25 deg model output, Gulf of Mexi...
    time_coverage_end:          2014-08-30T00:00:00Z
    time_coverage_start:        2009-04-02T00:00:00Z
    title:                      NRL HYCOM 1/25 deg model output, Gulf of Mexi...
    Westernmost_Easting:        -98.0